# Checking the ANN

In [ ]:
from keras.models import load_model
import dnns
import numpy as np
from dnns.noisy_sgd import NoisySGD


In [ ]:
# load model
model = load_model('trained_model_of_lenet_300_100_relu_crossent_noisy_sgd_retest_final_dense.h5', custom_objects={'NoisySGD':NoisySGD})

In [ ]:
# summarize model.
model.summary()
# load dataset

In [ ]:
from dnns.load_dataset import load_and_preprocess_dataset
is_output_categorical = True
dataset_info = load_and_preprocess_dataset(
    'mnist', categorical_output=is_output_categorical)
x_train, y_train = dataset_info['train']
x_test, y_test = dataset_info['test']
img_rows, img_cols = dataset_info['img_dims']
input_shape = dataset_info['input_shape']
num_classes = dataset_info['num_classes']

# reshape input to flatten data
x_train = x_train.reshape(x_train.shape[0], 1, np.prod(x_train.shape[1:]))
np.savez('x_train', x_train)
x_test = x_test.reshape(x_test.shape[0], 1, np.prod(x_test.shape[1:]))
np.savez('x_test', x_test)
x_norm = x_train[:10000].reshape(10000, 1, np.prod(x_train.shape[1:]))
np.savez('x_norm', x_norm)

np.savez('y_train', y_train)
np.savez('y_test', y_test)

In [ ]:
model.layers[1].data_format

In [ ]:

score = model.evaluate(x_test, y_test, verbose=1, batch_size=10)
print('Test Loss:', score[0])
print('Test Accuracy:', score[1])


# Running the Toolbox

In [ ]:
from snntoolbox.bin.run import run_toolbox

In [ ]:
config = """[paths]
dataset_path = %(path_wd)s
filename_ann = trained_model_of_lenet_300_100_relu_crossent_noisy_sgd_retest_final_dense
runlabel = lenet_dense

[tools]
evaluate_ann = False
normalize = True
only_serialise = True

[normalization]
percentile = 90.0

[simulation]
simulator = spiNNaker
duration = 200
num_to_test = 5
batch_size = 5
keras_backend = tensorflow
dt = 0.1

[cell]
tau_syn_E = 0.2
tau_syn_I = 0.2
v_thresh = 1

[input]
poisson_input = True


[output]
log_vars = {'all'}
plot_vars = {'all'}"""

file='config'
with open(file, 'w') as filetowrite:
    filetowrite.write(config)

f = open("config","r")

t_stim = 200
testing_examples = 5

In [ ]:
#necessary to fully reset simulator
from spinn_front_end_common.utilities import globals_variables
globals_variables.unset_simulator()

run_toolbox(config_filepath='config',terminal=True)
#complains because it tries to do data analysis
#in future it would be better to put serialisation inside the toolbox

# Running the serialised model

In [ ]:
#necessary to fully reset simulator
from spinn_front_end_common.utilities import globals_variables
globals_variables.unset_simulator()

extra_args = ['lenet_dense_serialised', '--t_stim', str(t_stim), '--testing_examples',\
              str(testing_examples), '--result_filename', 'output_data', '--result_dir', '.']
import pynn_object_serialisation.experiments.mnist_testing.mnist_testing as mnist_testing
from pynn_object_serialisation.experiments.mnist_testing.mnist_argparser import parser

new_args = parser.parse_args(extra_args) 
mnist_testing.run(new_args)


# Looking at the output

In [ ]:
from pynn_object_serialisation.OutputDataProcessor import OutputDataProcessor

In [ ]:
output = OutputDataProcessor('output_data.npz')

In [ ]:
output.y_pred

In [ ]:
output.y_test

In [ ]:
output.accuracy

In [ ]:
output.y_pred == output.y_test

In [ ]:
output.plot_bin(0, output.layer_names[3], shape = (28,28))

In [ ]:
output.layer_names

In [ ]:
output.get_bounds(1)

In [ ]:
output.t_stim

In [ ]:
output.dt

In [ ]:
output.delay